- Analyses comparing the interaction between traits, responses and fMRI beta slope regressors

In [ ]:
import numpy as np
import scipy as sp
from scipy import stats
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

plt.rcParams.update({'font.size': 16})

Read in restricted behavioral data:

In [ ]:
res_behav_data = pd.read_csv('../data/RESTRICTED_esfinn_11_21_2021_19_19_35.csv')
res_behav_data.set_index("Subject", inplace=True)
res_behav_data.index = res_behav_data.index.map(str)
print(res_behav_data.shape)
res_behav_data.head()

Read in unrestricted behavioral data:

In [ ]:
unres_behav_data = pd.read_csv('../data/unrestricted_esfinn_11_21_2021_19_19_13.csv')
unres_behav_data.set_index("Subject", inplace=True)
unres_behav_data.index = unres_behav_data.index.map(str)
print(unres_behav_data.shape)
unres_behav_data.head()

Combine restricted and unrestricted behavioral data into a single dataframe:

In [ ]:
behav_data = pd.concat([res_behav_data, unres_behav_data], axis=1)
print(behav_data.shape)
behav_data.head()

Read in FMRI data, join into single dataframe, then join this with the larger dataframe:

In [ ]:
data_file_loc = '/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/fMRIScipts/data'
sub_id_all = np.load(os.path.join(data_file_loc,'sub_ID_all.npy'))
sub_id_all = [str(i) for i in sub_id_all]
print(len(sub_id_all),sub_id_all[:5])

In [ ]:
# load behavioral responses too - to identify missed/invalid trials later - 1 for "social", 0 for "nonsocial" and 9 for "unsure", nan for missed response
responses = np.load(os.path.join(data_file_loc,'responses.npy'))
responses.shape # subs *movies

In [ ]:
# load slope regression coefficents for the fMRI data
coef_slope_rand = np.load(os.path.join(data_file_loc,'coefs_run_norm','slope_reg','coef_slopereg_runnorm_Random mechanical.npy'))
coef_slope_fish = np.load(os.path.join(data_file_loc,'coefs_run_norm','slope_reg','coef_slopereg_runnorm_Fishing.npy'))
coef_slope_coax = np.load(os.path.join(data_file_loc,'coefs_trial_norm','slope_reg','coef_slopereg_trialnorm_COAXING-B.npy'))
coef_slope_bill = np.load(os.path.join(data_file_loc,'coefs_trial_norm','slope_reg','coef_slopereg_trialnorm_BILLIARD-A.npy'))
coef_slope_all  = np.load(os.path.join(data_file_loc,'coef_slopereg_all.npy'))

print(coef_slope_rand.shape)
coef_slope_all.shape

In [ ]:
shen268_lbl = pd.read_csv(os.path.join(data_file_loc,"shen_dictionary.csv"))
pd.set_option('display.max_colwidth', None)
shen268_lbl.head()

In [ ]:
# for comparison with the other main figure if needed
nodes_coaxbill_rand_all = np.load(os.path.join(data_file_loc,'nodes_coaxbill_rand_all.npy')) # sig nodes hihglighted in the first S>NS GLM
len(np.where(nodes_coaxbill_rand_all)[0])

## Directly compare "Social" and "Non-social" using a linear regression for RANDOM MECH (and maybe FISHING)


Procedure:
1. compare the models: bold = b0+b1*response, bold = b0 + b1*response + b2*traits + b3*traits*responses, and bold = b0 + b1*response + b2*traits + b3*traits*responses
2. select nodes for which the second model (or third depending on what we're looking for) is better AIC-wise
3. extract coeffts for these nodes alone (redo another LR just for these nodes), fdr-correct and check nodes for significance.


In [ ]:
from multipy.fdr import lsu
from pymer4.models import Lm

In [ ]:
## RANDOM MECH
# find out in which nodes AIC decreases with trait terms
# model 1: factor: responses
# model 2: factor: responses, traits (no interactions)
# model 3: factors: traits, responses, their interaction

coef_nodes_rand_lr_model1, pval_nodes_rand_lr_model1 = [np.empty((268, )) for _ in range(2)]
coef_nodes_rand_lr_model2, pval_nodes_rand_lr_model2 = [np.empty((268,2)) for _ in range(2)]
coef_nodes_rand_lr_model3, pval_nodes_rand_lr_model3 = [np.empty((268,3)) for _ in range(2)]
coef_nodes_rand_lr_model1[:],pval_nodes_rand_lr_model1[:],coef_nodes_rand_lr_model2[:],pval_nodes_rand_lr_model2[:],coef_nodes_rand_lr_model3[:],pval_nodes_rand_lr_model3[:]  = [np.nan]*6

aic_nodes_rand_lr = np.empty((268,3))
aic_nodes_rand_lr[:] = np.nan

for n in range(268): 
    if n% 50 ==0:
        print('node',n)
    fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_node':coef_slope_rand[:,n], 'responses':responses[:,4]})
    fMRI_data.set_index("Subject", inplace=True)
    data = fMRI_data.join(behav_data, how='inner') # join betas and trait info
    data.reset_index(inplace=True)
    data = data.loc[:,['Subject','slopeReg_node','responses','ASR_Intn_T']]
    
    d = data.loc[(data['responses']==0)|(data['responses']==1),:]
    ind = (~np.isnan(d['slopeReg_node'])) & (~np.isnan(d['ASR_Intn_T'])) & (~np.isnan(d['responses']))
    d = d.loc[ind,:]
    
    model1 = Lm("slopeReg_node ~ responses", data=d)
    model1.fit(summary = False, verbose = False)
    
    model2 = Lm("slopeReg_node ~ responses + ASR_Intn_T", data=d)
    model2.fit(summary = False, verbose = False)

    model3 = Lm("slopeReg_node ~ responses + ASR_Intn_T + responses*ASR_Intn_T", data=d)
    model3.fit(summary = False, verbose = False)
    #model.fit(factors={"response":["Social","Unsure"]},summary = False, verbose = False) # Unsure > Social
   
    if n == 0:
        print(model1.fit())#factors={"response":["Social","Unsure"]}))
        print(model2.fit())
        print(model3.fit())

    if (len(model1.warnings) == 0):
        coef_nodes_rand_lr_model1[n] = model1.coefs['Estimate'][1]
        pval_nodes_rand_lr_model1[n] = model1.coefs['P-val'][1]
        aic_nodes_rand_lr[n,0] = model1.AIC

    if (len(model2.warnings) == 0):
        for ind in range(2):
            coef_nodes_rand_lr_model2[n,ind] = model2.coefs['Estimate'][ind+1]
            pval_nodes_rand_lr_model2[n,ind] = model2.coefs['P-val'][ind+1]
        aic_nodes_rand_lr[n,1] = model2.AIC

    if (len(model3.warnings) == 0):
        for ind in range(3):
            coef_nodes_rand_lr_model3[n,ind] = model3.coefs['Estimate'][ind+1]
            pval_nodes_rand_lr_model3[n,ind] = model3.coefs['P-val'][ind+1]
        aic_nodes_rand_lr[n,2] = model3.AIC

pval_nodes_fdr_rand_lr_model1 = lsu(pval_nodes_rand_lr_model1,q=.05)

pval_nodes_fdr_rand_lr_model2 = np.empty_like(pval_nodes_rand_lr_model2)
pval_nodes_fdr_rand_lr_model2[:]=np.nan
for ind in range(2):
    pval_nodes_fdr_rand_lr_model2[:,ind] = lsu(pval_nodes_rand_lr_model2[:,ind],q=.05)

pval_nodes_fdr_rand_lr_model3 = np.empty_like(pval_nodes_rand_lr_model3)
pval_nodes_fdr_rand_lr_model3[:]=np.nan
for ind in range(3):
    pval_nodes_fdr_rand_lr_model3[:,ind] = lsu(pval_nodes_rand_lr_model3[:,ind],q=.05)


In [ ]:
aic_nodes_rand_lr1 = np.empty_like(aic_nodes_rand_lr)
for i in [2,1,0]:
    aic_nodes_rand_lr1[:,i] = aic_nodes_rand_lr[:,i]- aic_nodes_rand_lr[:,0]
    #aic_nodes_corr_fish_lr[:,i] = aic_nodes_corr_fish_lr[:,i]- aic_nodes_corr_fish_lr[:,0]

plt.figure(1,figsize=(5,10))
plt.plot(aic_nodes_rand_lr1.T)
plt.xticks(range(3),['only response','response+\ntraits','response+\ntraits+\ninteraxn'])
plt.hlines(-2,0,2,'grey',ls='dotted')
plt.ylabel('AIC - \nAIC(reponse-only model)')
plt.xlabel('models (simple to complex -->)')
plt.title('RANDOM MECH')


In [ ]:
rand_nodes_add_traits = np.where((aic_nodes_rand_lr1[:,1]<-2))[0] #  response only v. trait main
print(len(nodes_traits),rand_nodes_add_traits)

rand_nodes_add_traits_interaxn = np.where(aic_nodes_rand_lr1[:,2]<-2)[0] #  trait main v. trait main+int
print(len(nodes_traits),rand_nodes_add_traits_interaxn)

In [ ]:
# response only v. responses+ traits

def get_sig_nodes(pval,nodes):
    try:
        terms =  pval.shape[1]
    except:
        terms = 1
        pval = pval[:,np.newaxis]
    #print(terms)
    all_nodes =  np.empty((268,terms))
    all_nodes[:] = np.nan
    for i in range(terms):
        all_nodes[nodes,i] = pval[nodes,i]
    return [['term '+str(i+1),np.where(all_nodes[:,i]==1)[0]] for i in range(terms)]

print('FDR-corr. response-only vs. responses+traits')
print('response only:')
print(get_sig_nodes(pval_nodes_fdr_rand_lr_model1, rand_nodes_add_traits))
print('\nresponse + traits:')
print(get_sig_nodes(pval_nodes_fdr_rand_lr_model2, rand_nodes_add_traits))

# response only v. responses+ traits + interaction
'''print('\n\nFDR-corr. response-only vs. responses+traits+interaction')
print('response only:')
print(get_sig_nodes(pval_nodes_fdr_rand_lr_model1,rand_nodes_add_traits_interaxn))
print('\nresponse + traits + interactionn:')
get_sig_nodes(pval_nodes_fdr_rand_lr_model3,rand_nodes_add_traits_interaxn)'''

- So no significat term for 'traits' at the corrected level, and we get one additional sig. term for  'responses' - prob because it improves the model fit 
- How about at the uncorrected thresholds?


In [ ]:
#response only v. responses+ traits
print('Unc. response-only vs. responses+traits')
print('response only:')
print(get_sig_nodes(pval_nodes_rand_lr_model1<.05,rand_nodes_add_traits))
print('\nresponse + traits:')
print(get_sig_nodes(pval_nodes_rand_lr_model2<.05,rand_nodes_add_traits))

# response only v. responses+ traits + interaction
'''print('\n\nUnc. response-only vs. responses+traits+interaction')
print('response only:')
print(get_sig_nodes(pval_nodes_rand_lr_model1<.05,rand_nodes_add_traits_interaxn))
print('\nresponse + traits + interactionn:')
get_sig_nodes(pval_nodes_rand_lr_model3<.05,rand_nodes_add_traits_interaxn)'''


- There are several nodes showing a significant 'traits' term but at the unc. threshold

In [ ]:
# unc. threshold so don't take to heart. what are these regions though?
for n in rand_nodes_add_traits:
    print(n+1,eval(shen268_lbl[str(n+1)][0])['name'])


- mostly brainstem regions. Maybe more power with ALL MOVIES?

# ALL MOVIES : Directly compare "Social" and "Non-social" using lr for ALL MOVIES


Procedure:
1. compare the models: bold = b0+b1*response, bold = b0 + b1*response + b2*traits + b3*traits*responses, and bold = b0 + b1*response + b2*traits + b3*traits*responses - subject as random factor in all models
2. select nodes for which the second model (or third depending on what we're looking for) is better AIC-wise
3. extract coeffts for these nodes alone (redo another LR just for these nodes), fdr-correct and check nodes for significance.

In [ ]:
# first of all, best to remove subjects with < 10 responses for power in general
count_resp = np.zeros((responses.shape[0],))
for i in range(responses.shape[0]):
    count_resp[i] = len(np.where(~np.isnan(responses[i,:]))[0])
subs_10resp = np.where(count_resp == 10)[0]
len(subs_10resp)

In [ ]:
# creating a 3D array of beta coeffs across movies from individual movie files
all_coefs = np.zeros((1048,268,10))

vidnames = ["COAXING-B", "BILLIARD-A", "DRIFTING-A", "Fishing", "Random mechanical","Scaring", "SEDUCING-B", "STAR-A", "SURPRISING-B", "TENNIS-A"]
for m in range(10):
    fileName =  os.path.join(data_file_loc,'coefs_run_norm','slope_reg',f'coef_slopereg_runnorm_{vidnames[m]}.npy')
    all_coefs[:,:,m] = np.load(fileName)
    #print(dat.shape)

In [ ]:
from pymer4.models import Lm
from multipy.fdr import lsu
#from pymer4.models import Lmer
import time

In [ ]:
# same analysis as above, all nodes
# ALL MOVIES

coef_nodes_all_model1,   pval_nodes_all_model1    = [np.empty((268,)) for _ in range(2)]
coef_nodes_all_model1[:],pval_nodes_all_model1[:] = [np.nan] *2

coef_nodes_all_model2,   pval_nodes_all_model2    = [np.empty((268,2)) for _ in range(2)]
coef_nodes_all_model2[:],pval_nodes_all_model2[:] = [np.nan] *2

coef_nodes_all_model3,   pval_nodes_all_model3    = [np.empty((268,3)) for _ in range(2)]
coef_nodes_all_model3[:],pval_nodes_all_model3[:] = [np.nan] *2

aic_nodes_all_lme = np.empty((268,3))
aic_nodes_all_lme[:] = np.nan

pval_nodes_fdr_all_model1 = np.empty_like(pval_nodes_all_model1)
pval_nodes_fdr_all_model1[:] = np.nan

pval_nodes_fdr_all_model2 = np.empty_like(pval_nodes_all_model2)
pval_nodes_fdr_all_model2[:] = np.nan

pval_nodes_fdr_all_model3 = np.empty_like(pval_nodes_all_model3)
pval_nodes_fdr_all_model3[:] = np.nan

nresp = [len(np.where(~np.isnan(responses[i,:]))[0]) for i in range(responses.shape[0])]
start_time = time.time()
for n in range(268):
    if n% 50 ==0:
        print(f'node_ind  {n}, time taken: {time.time()-start_time:.2f}')
    
    data = pd.DataFrame({})#columns = ['Subject','coef','response','movie'])
    for m in range(10):
        #fMRI_data = pd.DataFrame({'coefs': all_coefs[subs_10resp,n,m],'response':responses[subs_10resp,m], 'subID':sub_id_all[subs_10resp], 'movie':np.repeat(m,len(subs_10resp))})
        fMRI_data = pd.DataFrame({'Subject':sub_id_all,'coef': all_coefs[:,n,m], 'response': responses[:,m],'nresp':nresp, 'movie': np.repeat(m,1048)})
        fMRI_data = fMRI_data.loc[fMRI_data['nresp']==10,:]
        fMRI_data.set_index("Subject", inplace=True)
        data_temp = fMRI_data.join(behav_data['ASR_Intn_T'], how='inner') # join betas and trait info
        data = data.append(data_temp)#,ignore_index=True)
    data.reset_index(inplace=True)
    #if n == 0:
    #    print(data.shape)
    #    print(data.head())
    
    #d = data.loc[data['responses']==10,:]
    inds = ~np.isnan(data['coef']) & ~np.isnan(data['ASR_Intn_T']) & (data['response']!=9) # find rows where neither x or y is NaN
    data = data.loc[inds,:]
    # Calculate and print correlations
    
    #model1 = Lmer('coef ~  response + (1|movie)', data=data) # ff: mean response, rf:subjID
    model1 = Lm('coef ~  response', data=data) # ff: mean response, rf:subjID
    model1.fit(summary = False, verbose = False, no_warnings = True)
    #model.fit(factors={"response":["Non-social","Social"]},summary = False, verbose = False)

    #model2 = Lmer('coef ~  response + ASR_Intn_T + (1|movie)', data=data) # ff: mean response, rf:subjID
    model2 = Lm('coef ~  response + ASR_Intn_T', data=data) # ff: mean response, rf:subjID
    model2.fit(summary = False, verbose = False, no_warnings = True)

    #model3 = Lmer('coef ~  response + ASR_Intn_T + response*ASR_Intn_T + (1|movie) ', data=data) # ff: mean response, rf:subjID
    model3 = Lm('coef ~  response + ASR_Intn_T + response*ASR_Intn_T', data=data) # ff: mean response, rf:subjID
    model3.fit(summary = False, verbose = False, no_warnings = True)

    #model.fit(summary = False, verbose = False)
    if (len(model1.warnings) == 0):
        coef_nodes_all_model1[n] = model1.coefs['Estimate'][1]
        pval_nodes_all_model1[n]= model1.coefs['P-val'][1]
        aic_nodes_all_lme[n,0] = model1.AIC

    if (len(model2.warnings) == 0):
        for ind in range(2):
            coef_nodes_all_model2[n,ind] = model2.coefs['Estimate'][ind+1]
            pval_nodes_all_model2[n,ind] = model2.coefs['P-val'][ind+1]
        aic_nodes_all_lme[n,1] = model2.AIC

    if (len(model3.warnings) == 0):
        for ind in range(3):
            coef_nodes_all_model3[n,ind] = model3.coefs['Estimate'][ind+1]
            pval_nodes_all_model3[n,ind] = model3.coefs['P-val'][ind+1]
        aic_nodes_all_lme[n,2] = model3.AIC

pval_nodes_fdr_all_model1 = lsu(pval_nodes_all_model1,q=.05)

for ind in range(2):
    pval_nodes_fdr_all_model2[:,ind] = lsu(pval_nodes_all_model2[:,ind],q=.05)

for ind in range(3):
    pval_nodes_fdr_all_model3[:,ind] = lsu(pval_nodes_all_model3[:,ind],q=.05)
 

In [ ]:
aic_nodes_all_lme1 = np.empty((268,3))
aic_nodes_all_lme1[:] = np.nan
for i in [2,1,0]:
    aic_nodes_all_lme1[:,i] = aic_nodes_all_lme[:,i]- aic_nodes_all_lme[:,0]


In [ ]:
plt.figure(1,figsize=(5,10))
plt.plot(aic_nodes_all_lme1.T)
plt.xticks(range(3),['only response','response+\ntraits','response+\ntraits+\ninteraxn'])
plt.hlines(-2,0,2,'grey',ls='dotted')
plt.ylabel('AIC - \nAIC(reponse-only model)')
plt.xlabel('models (simple to complex -->)')
plt.title('ALL MOVIES (linear regressions)')


In [ ]:
allmovies_nodes_add_traits = np.where((aic_nodes_all_lme1[:,1]<-2))[0] #  response only v. trait main
print(len(nodes_traits),allmovies_nodes_add_traits)

allmovies_nodes_add_traits_interaxn = np.where(aic_nodes_all_lme1[:,2]<-2)[0] #  trait main v. trait main+int
print(len(nodes_traits),allmovies_nodes_add_traits_interaxn)


In [ ]:
def intersection(lst1, lst2):
    
    if type(lst1) != 'list':
        lst1 = list(lst1)
    
    if type(lst2) != 'list':
        lst2 = list(lst2)
    
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

intersection (allmovies_nodes_add_traits, rand_nodes_add_traits)

In [ ]:
print('FDR-corr. response-only vs. responses+traits')
print('response only:')
print(get_sig_nodes(pval_nodes_fdr_all_model1, allmovies_nodes_add_traits))
print('\nresponse + traits:')
print(get_sig_nodes(pval_nodes_fdr_all_model2, allmovies_nodes_add_traits))


#response only v. responses+ traits
print('\n\nUnc. response-only vs. responses+traits')
print('response only:')
print(get_sig_nodes(pval_nodes_all_model2<.05,allmovies_nodes_add_traits))
print('\nresponse + traits:')
print(get_sig_nodes(pval_nodes_all_model2<.05,allmovies_nodes_add_traits))


## COAX-BILL

- If coax-bill or all movies overlap with RANDOM MECH at least, what we did makes sense - too weak otherwise

In [ ]:
# same analysis as above, all nodes
# COAX-BILL
corrtype = 'Spearman' # 'Pearson'
x = "slopeReg_diff"
#y = "Loneliness_Unadj"
y = "ASR_Intn_T"

corr_nodes_coaxbill,   pval_nodes_coaxbill    =  [np.empty((268,)) for _ in range(2)]
corr_nodes_coaxbill[:],pval_nodes_coaxbill[:] = [np.nan] *2

pval_nodes_fdr_coaxbill = np.empty_like(pval_nodes_coaxbill)
pval_nodes_fdr_coaxbill[:] = np.nan

sig_nodes = np.where(nodes_coaxbill_rand_all)[0] # nodes sig in the main GLM analysis

for n in sig_nodes:
#for n in range(268): 
    #if n% 50 ==0:
    print('node',n)

    fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_diff':coef_slope_coax[:,n]-coef_slope_bill[:,n], 'responses':(responses[:,0]==1)&(responses[:,1]==0)})
    fMRI_data.set_index("Subject", inplace=True)
    data = fMRI_data.join(behav_data, how='inner') # join betas and trait info
    #if n == 0:
    #    print(data.shape)
    #    print(data.head())
    
    d = data.loc[data['responses']==True,:]
    inds = ~np.isnan(d[x]) & ~np.isnan(d[y]) # find rows where neither x or y is NaN
    
    # Calculate and print correlations
    if corrtype == 'Pearson':
        corr_nodes_coaxbill[n], pval_nodes_coaxbill[n] = stats.pearsonr(d[x][inds], d[y][inds])
    elif corrtype =='Spearman':
        corr_nodes_coaxbill[n], pval_nodes_coaxbill[n] = stats.spearmanr(d[x][inds], d[y][inds])

pval_nodes_fdr_coaxbill[sig_nodes] = lsu(pval_nodes_coaxbill[sig_nodes],q=.05)
pval_ = lsu(pval_nodes_coaxbill[sig_nodes],q=.05)



In [ ]:
print('Nodes sig. at the corrected threshold:', np.where(pval_nodes_fdr_coaxbill[sig_nodes]))
print('Nodes sig. at the unc. threshold:',sig_nodes[pval_nodes_coaxbill[sig_nodes] <.05],',p-values:',pval_nodes_coaxbill[sig_nodes][pval_nodes_coaxbill[sig_nodes] <.05])

In [ ]:
# intersection with the other analyses?

print(intersection(sig_nodes[pval_nodes_coaxbill[sig_nodes] <.05],rand_nodes_add_traits))
print(intersection(sig_nodes[pval_nodes_coaxbill[sig_nodes] <.05],rand_nodes_add_traits_interaxn))
print(intersection(sig_nodes[pval_nodes_coaxbill[sig_nodes] <.05],allmovies_nodes_add_traits))
print(intersection(sig_nodes[pval_nodes_coaxbill[sig_nodes] <.05],allmovies_nodes_add_traits_interaxn))

[eval(shen268_lbl[str(n+1)][0])['name'] for n in [68,208]]

In [ ]:
print('COAX-BILL')
print('p corr. "Social"- "Non-social" responders',np.where(pval_nodes_corr_coaxbill))
print('p unc. "Social" - "Non-social" responders',np.where(pval_nodes_coaxbill<.05))
print('rand:',rand_sig_interaction_nodes[:,0])


# Any overlap between RAND and COAX-BILL?
print('\nnodes where diff between correlation are sig in both COAX-BILL and RAND (linreg):',intersection(rand_sig_interaction_nodes[:,0],np.where(pval_nodes_coaxbill<.05)[0]))
#print('nodes where diff between correlation are sig in both COAX-BILL and RAND:',np.where((pCorrs_rand<.05)          & (pval_nodes_coaxbill<.05))[0])
#print('nodes where "Social" correlations are sig in both COAX-BILL and RAND:',   np.where((pval_nodes_rand[:,1]<.05) & (pval_nodes_coaxbill<.05))[0])



In [ ]:
intersection(np.where(nodes_coaxbill_rand_all)[0],np.where(pval_nodes_coaxbill<.05)[0])

# ALL MOVIES - run-wise estimates

In [ ]:
print('ALL MOVIES')

print('p corr. "Social"- "Non-social" responders',np.where(pval_nodes_corr_all))
print('p unc. "Social" - "Non-social" responders',np.where(pval_nodes_all<.05))
print('rand:',rand_sig_interaction_nodes[:,0])



nmin,nmax = np.argmin(corr_nodes_coaxbill),np.argmax(corr_nodes_coaxbill)
print(f'coaxbill, Node {nmin+1}, rSp={corr_nodes_coaxbill[nmin]:.2f}, pSp = {pval_nodes_coaxbill[nmin]:.2e}')
print(f'coaxbill, Node {nmax+1}, rSp={corr_nodes_coaxbill[nmax]:.2f}, pSp = {pval_nodes_coaxbill[nmax]:.2e}')


nmin,nmax = np.argmin(corr_nodes_all),np.argmax(corr_nodes_all)
print(f'\n\nall movies, Node {nmin+1}, rSp={corr_nodes_coaxbill[nmin]:.2f}, pSp = {pval_nodes_coaxbill[nmin]:.2e}')
print(f'all movies, Node {nmax+1}, rSp={corr_nodes_coaxbill[nmax]:.2f}, pSp = {pval_nodes_coaxbill[nmax]:.2e}')

In [ ]:

tr0_ind = 3 # ind at which tr=0 starts
[timepts_indiv_movie,vid_start_rel_tr] = np.load(os.path.join(data_file_loc,'Video_TRs.npy'),allow_pickle=True)
print(vid_start_rel_tr)
l_task = 28

def remove_pretrial_TRs(tcs,vid_start_rel_tr,pretrial_TRs):

    tcs1 = np.empty((tcs.shape[0],tcs.shape[1],28+pretrial_TRs,tcs.shape[3]))
    tcs1[:] = np.nan

    for vid_no in range(10):
        tr0 = vid_start_rel_tr[vid_no]
        tcs1[:,:,:,vid_no] = tcs[:,:,tr0-pretrial_TRs:tr0+28,vid_no] # 31 timepts
    return tcs1


print('\ntrial-wise norm data.')
fileName = os.path.join(data_file_loc,'timecourses_trial_norm','timecourse-all-movies_zscorenorm.npy')
tcs_trial_z = np.load(fileName) # nsubs * nnodes * ntimepts *nmovies
print('before:',tcs_trial_z.shape)
tcs_trial_z = remove_pretrial_TRs(tcs_trial_z,vid_start_rel_tr,tr0_ind)
print('after:',tcs_trial_z.shape)


In [ ]:
left_rois = pd.read_excel(os.path.join('/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/paper_prep/figures/fig5_timecourse/ROIs_left.xls'))
left_rois

In [ ]:
right_rois = pd.read_excel(os.path.join('/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/paper_prep/figures/fig5_timecourse/ROIs_right.xls'))
right_rois

In [ ]:
lr_rois = pd.concat([left_rois,right_rois])
lr_rois

In [ ]:
TR = .72
tr = [int(i) for i in lr_rois['Divergence\n(sec)']/TR]
tc_analysis_nodes = np.array([int(i) for i in lr_rois['Shen atlas node\n(1-268)'].values - 1])
tc_analysis_nodes

In [ ]:
traitdep_nodes_coaxbill = np.where(pval_nodes_coaxbill <.05)[0]
print(traitdep_nodes_coaxbill)
intersection_nodes_coaxbill = intersection(tc_analysis_nodes,traitdep_nodes_coaxbill )
intersection_nodes_coaxbill

# Comparing activity at divergence pt

In [ ]:
def explore_correlation(x, y, data, str, ax, corrType = 'Spearman'):
    """
    Calculates and plots correlation between x and y variables in dataframe `data`, plus distribution of x and y 
    """
    sns.set_style("white")
    
    inds = ~np.isnan(data[x]) & ~np.isnan(data[y]) # find rows where neither x or y is NaN

    #g = sns.jointplot(x=x, y=y, data=data, kind='reg', color='gray')
    #g = sns.regplot(x=x, y=y, data=data, color='gray')
    g = sns.regplot(x=x, y=y, data=data, color='gray')
    
    xmin,xmax = -3,3
    plt.xlim(xmin,xmax)
    ymin,ymax = 0,100
    plt.ylim(ymin,ymax)
        
    # Calculate and print correlations
    if corrType == 'Pearson':
        rp, pp = stats.pearsonr(data[x][inds], data[y][inds])
    else:
        rs, ps = stats.spearmanr(data[x][inds], data[y][inds])
    #g.ax_joint.annotate(f'{str},r_s = {rs:.2f} (p={ps:.1g})', xy=(.05,.95), xycoords='axes fraction')
    g.annotate(f'{str},r_s = {rs:.2f} (p={ps:.1g})', xy=(.05,.95), xycoords='axes fraction')
    
    return g

In [ ]:
x = "bold_diff"
#y = "Loneliness_Unadj"
y = "ASR_Intn_T"

tr0_ind=3

corr_nodes_coaxbill_divpt,pval_nodes_coaxbill_divpt = [np.empty((268,)) for _ in range(2)]
corr_nodes_coaxbill_divpt[:],pval_nodes_coaxbill_divpt[:] = [np.nan] *2

for n in intersection_nodes_coaxbill:
    timept = tr0_ind +  tr[int(np.where(tc_analysis_nodes==n)[0])] #tr[tc_analysis_nodes==n]
    bold_diff= tcs_trial_z[:,n,timept,0] - tcs_trial_z[:,n,timept,1]
    fMRI_data = pd.DataFrame({'Subject':sub_id_all,'bold_diff':bold_diff, 'responses':(responses[:,0]==1)&(responses[:,1]==0)})
    fMRI_data.set_index("Subject", inplace=True)
    data = fMRI_data.join(behav_data, how='inner') # join betas and trait info
    #if n == 0:
    #    print(data.shape)
    #    print(data.head())
    
    d = data.loc[data['responses']==True,:]
  
    #if n == 0:
    #    print(data.shape)
    #    print(data.head())
    
    #inds = ~np.isnan(d[x]) & ~np.isnan(d[y]) # find rows where neither x or y is NaN
    
    explore_correlation(x,y,d,'Spearman')
    plt.text(-2,100,f'Node{n+1}',fontweight='bold')
    # Calculate and print correlations
    '''if corrtype == 'Pearson':
        corr_nodes_coaxbill_divpt[n], pval_nodes_coaxbill_divpt[n] = stats.pearsonr(d[x][inds], d[y][inds])
    elif corrtype =='Spearman':
        corr_nodes_coaxbill_divpt[n], pval_nodes_coaxbill_divpt[n] = stats.spearmanr(d[x][inds], d[y][inds])
'''

- the above (lack of) correlations show that the differences at the divergence pt does not directly depend on trait scores

In [ ]:
#tc_analysis_nodes = {'right': [65,68,69,75,80], 'left': [196,197,198,205,206,209,212,213]}


# Plot brain maps

In [ ]:
from nltools.data import Brain_Data
from nltools.mask import expand_mask, roi_to_brain
from nilearn.plotting import plot_glass_brain, plot_surf_roi,plot_stat_map,plot_img,plot_surf_contours
from nilearn import datasets
from nilearn.surface import vol_to_surf

import nibabel as nib

from nilearn.datasets import fetch_surf_fsaverage
fsaverage = fetch_surf_fsaverage()

#coords = [10*int(i) for i in np.linspace(-4,7,6)]
bg_img = datasets.load_mni152_template()

mask = Brain_Data('https://neurovault.org/media/images/8423/shen_2mm_268_parcellation.nii.gz')
mask_x = expand_mask(mask)


In [ ]:
def color_rois(values):
    
    """
    This function assumes you are passing a vector "values" with the same length as the number of nodes in the atlas.
    """
    shen268 = nib.load(os.path.join(data_file_loc,"shen_2mm_268_parcellation.nii.gz"))
    shen268_data = shen268.get_fdata()
    img = np.zeros(shen268_data.shape)
    #print(shen268_data.shape)
    for roi in range(len(values)):
        itemindex = np.where(shen268_data==roi+1) # find voxels in this node (add 1 to account for zero-indexing)
        #print(len(itemindex[0]))
        img[itemindex] = values[roi] # color them by the desired value 

    affine = shen268.affine
    img_nii = nib.Nifti1Image(img, affine)
    
    return img_nii

In [ ]:
fig_save_loc = os.path.join('/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/paper_prep/figures/fig2_2_traits/fmri_results')

# interaction plots


In [ ]:
coords = [-50,-40,-25,-10,0,10,25,40] # initial exploration
#coords = [-40,-30,-20] # for the final version
cmap = 'RdBu_r'

resp_lbl = ["Nonsocial",  "Social"]
    
vmin,vmax = -.03,.03

#txt = r"$\overline{\beta}(''Social''-''Non-social'')$"
txt = 'Interaction term\n(trait_slope"Social"-\ntrait_slope"Non-social")\n,q<.05'

nodes = np.zeros((268,))
node_inds = [int(i) for i in rand_sig_interaction_nodes[:,0]]
nodes[node_inds] = rand_sig_interaction_nodes[:,1]
title_txt = f'RANDOM MECH "Social"-"Non-social"'

fig,ax = plt.subplots(nrows=1,ncols=4,figsize = (20,5),subplot_kw={'projection': '3d'})


img = roi_to_brain(pd.Series(nodes), mask_x)
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = .0001)
ax_plot.add_contours(color_rois((nodes_coaxbill_rand_all)),linewidths=1, colors=[[.3,.3,.3,1]],linestyles='solid')
#ax_plot.add_contours(color_rois((pCorrs_rand<.05)),linewidths=2, colors=['g'],linestyles='solid')
#ax_plot.title(title_txt,fontsize=24,bgcolor='k',color='w',fontweight='bold')
#ax_plot._colorbar_ax.text(1,.8*vmax,txt,fontsize=20,fontdict = {'verticalalignment':'top','rotation':0})#get_legend()
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/axial.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

#LH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
surf_plot1=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_left,axes=ax[0])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})

#surf_plot1.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
texture_contour1 = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
plot_surf_contours(fsaverage.infl_left, texture_contour1, axes = ax[0], figure=surf_plot1, legend=True,levels = [1], colors=[[.3,.3,.3,1]])
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/left_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')   
#plt.clf()
   

surf_plot2=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_left, view = 'medial',axes=ax[1])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
plot_surf_contours(fsaverage.infl_left, texture_contour1, axes = ax[1], figure=surf_plot2, legend=True,levels = [1], colors=[[.3,.3,.3,1]])
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/left_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

#RH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right,axes=ax[3])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
surf_plot3.axes[4].text(.4,.5*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
texture_contour1 = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
plot_surf_contours(fsaverage.infl_right, texture_contour1, axes = ax[3],figure=surf_plot3, legend=True,levels = [1], colors=[[.3,.3,.3,1]])
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/right_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()


surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=False,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right, view ='medial',axes=ax[2])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
plot_surf_contours(fsaverage.infl_right, texture_contour1, axes = ax[2], figure=surf_plot4, legend=True,levels = [1], colors=[[.3,.3,.3,1]])
#plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/right_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()


In [ ]:
from nilearn.plotting import plot_img_on_surf
plot_img_on_surf(img.to_nifti(),threshold=.001,
                          views=['lateral', 'medial'],
                          hemispheres=['left', 'right'],
                          colorbar=True,cmap= cmap)

In [ ]:
order = np.argsort(-rand_sig_interaction_nodes[:,1]) # in descending order of the interaction coefft
node_inds = [int(i) for i in rand_sig_interaction_nodes[order,0]]


In [ ]:
x = "coef_slope"
#y = "Loneliness_Unadj"
y = "ASR_Intn_T"

tr0_ind=3

for i,n in enumerate(node_inds):
    fig,ax = plt.subplots(figsize=(10,5),nrows=1,ncols=2)
    hem = 'Left' if n >=134 else 'Right'
    ind = order[i]
    sl = np.round(1000*rand_sig_interaction_nodes[ind,1])/1000
    plt.suptitle('Node ' + str(n+1) + '(' + hem + ') ' +  "\n".join(eval(shen268_lbl[str(n+1)][0])["name"].split(';')))
       
    for resp in range(2):        
        respType = '"Social"' if resp == 1 else '"Non-social"'
        rows = [int(i) for i in np.where(responses[:,4]==resp)[0]]
        fMRI_data = pd.DataFrame({'Subject':np.array(sub_id_all)[rows],'coef_slope':coef_slope_rand[rows,n], 'responses':rows})
        fMRI_data.set_index("Subject", inplace=True)
        data = fMRI_data.join(behav_data, how='inner') # join betas and trait info
        
        #explore_correlation(x,y,d,ax[i,resp],f'Node{n+1},{respType}','Spearman')
        sns.regplot(data[x],data[y],ax=ax[resp],color = 'grey')
        inds = (~np.isnan(data[x])) & (~np.isnan(data[y]))
        r,p = stats.spearmanr(data.loc[inds,x],data.loc[inds,y])
        ax[resp].set_title(respType)
        if p>.05:
            ax[resp].text(-4,70,f'r={r:.2f}\n(p={p:.2f})')
        else:
            ax[resp].text(-4,70,f'r={r:.2f}\n(p={p:.1e})')
        ax[resp].set_xlim(-4,4)
        ax[resp].set_ylim(30,80)
        ax[resp].set_xticks([-3,-2,-1,0,1,2,3])
        ax[resp].set_yticks([30,40,50,60,70,80])

    plt.tight_layout()

# Separate social/nonsocial plots

In [ ]:
coords = [-50,-40,-25,-10,0,10,25,40] # initial exploration
#coords = [-40,-30,-20] # for the final version
cmap = 'RdBu_r'

resp_lbl = ["Nonsocial",  "Social"]
    
vmin,vmax = -.32,.32
for resp in [0,1]:
    if resp == 1:
        txt = f'  {corrtype} r\n  (p<.05 unc.)\n {y} vs  'r"$\overline{\beta}(''Social'')$"
    else:
        txt = f'  {corrtype} r\n  (p<.05 unc.)\n {y} vs  'r"$\overline{\beta}(''Non-social'')$"

    nodes = np.zeros((268,))
    nodes[pval_nodes_rand[:,resp]<.05] = corr_nodes_rand[pval_nodes_rand[:,resp]<.05,resp]

    title_txt = f'RANDOM MECH "{resp_lbl[resp]}"'

    img = roi_to_brain(pd.Series(nodes), mask_x)
    ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = .0001)
    ax_plot.add_contours(color_rois((nodes_coaxbill_rand_all)),linewidths=1, colors=[[.3,.3,.3,1]],linestyles='solid')
    ax_plot.add_contours(color_rois((pCorrs_rand<.05)),linewidths=2, colors=['g'],linestyles='solid')
    #ax_plot.title(title_txt,fontsize=24,bgcolor='k',color='w',fontweight='bold')
    #ax_plot._colorbar_ax.text(1,.8*vmax,txt,fontsize=20,fontdict = {'verticalalignment':'top','rotation':0})#get_legend()
    #plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/axial.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
    #plt.clf()

    fig,ax = plt.subplots(nrows=1,ncols=4,figsize = (20,5),subplot_kw={'projection': '3d'})

    #LH
    texture = vol_to_surf(color_rois(nodes), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
    surf_plot1=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_left,axes=ax[0])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    #surf_plot1.axes[1].text(3,.8*vmax,s= txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
    surf_plot1.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
    texture_contour1 = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
    plot_surf_contours(fsaverage.infl_left, texture_contour1, figure=surf_plot1,axes=ax[0], legend=True,levels = [1], colors=[[.3,.3,.3,1]])
    #plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/left_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')   
    #plt.clf()

    surf_plot2=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_left, view = 'medial',axes=ax[1])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    #surf_plot2.axes[1].text(3,.8*vmax,s= txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
    #surf_plot2.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
    plot_surf_contours(fsaverage.infl_left, texture_contour1, figure=surf_plot2, legend=True,levels = [1], colors=[[.3,.3,.3,1]],axes=ax[1])
    #plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/left_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
    #plt.clf()

    #RH
    texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
    surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_right,axes=ax[3])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    surf_plot3.axes[1].text(3,.8*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
    texture_contour1 = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
    plot_surf_contours(fsaverage.infl_right, texture_contour1, figure=surf_plot3, axes=ax[3], legend=True,levels = [1], colors=[[.3,.3,.3,1]])
    #plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/right_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
    #plt.clf()

    surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=False,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_right, view ='medial',axes=ax[2])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    plot_surf_contours(fsaverage.infl_right, texture_contour1, figure=surf_plot4, axes=ax[2], legend=True,levels = [1], colors=[[.3,.3,.3,1]])
    #plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/right_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
    #plt.clf()


In [ ]:
# COAX-BILL
coords = [-50,-40,-25,-10,0,10,25,40] # initial exploration
#coords = [-40,-30,-20] # for the final version

txt = f'  {corrtype} r\n  (p<.05 unc.)\n {y} vs  \n  'r"$\overline{\beta(''Social''-}$" + '\n  ' + r"$\overline{\beta(''Non-social'')}$"
title_txt = 'COAX-BILL'

nodes = np.zeros((268,))
nodes[pval_nodes_coaxbill<.05] = corr_nodes_coaxbill[pval_nodes_coaxbill<.05]
#vmin,vmax = -max(abs(nodes)),max(abs(nodes))
vmin,vmax = -.32,.32

cmap = 'RdBu_r'

img = roi_to_brain(pd.Series(nodes), mask_x)
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = .0001)
ax_plot.add_contours(color_rois((nodes_coaxbill_rand_all)),linewidths=1, colors=[[.3,.3,.3,1]],linestyles='solid')

fig,ax = plt.subplots(nrows=1,ncols=4,figsize = (20,5),subplot_kw={'projection': '3d'})

#LH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
surf_plot1=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_left,axes=ax[0])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#surf_plot1.axes[1].text(1,.8*vmax,s= txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
surf_plot1.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
texture_contour = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
plot_surf_contours(fsaverage.infl_left, texture_contour, figure=surf_plot1,axes=ax[0], legend=True,levels = [1], colors=[[.3,.3,.3,1]])
#plt.savefig(os.path.join(fig_save_loc,f'COAXBILL/left_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

surf_plot2=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_left, view = 'medial',axes=ax[1])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#surf_plot2.axes[1].text(3,.8*vmax,s= txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
#surf_plot2.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
plot_surf_contours(fsaverage.infl_left, texture_contour, figure=surf_plot2,axes=ax[1], legend=True,levels = [1], colors=[[.3,.3,.3,1]])
#plt.savefig(os.path.join(fig_save_loc,f'COAXBILL/left_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

#RH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right,axes=ax[3])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
surf_plot3.axes[4].text(3,.8*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
#surf_plot3.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
texture_contour = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
plot_surf_contours(fsaverage.infl_right, texture_contour, figure=surf_plot3,axes=ax[3], legend=True,levels = [1], colors=[[.3,.3,.3,1]])
#plt.savefig(os.path.join(fig_save_loc,f'COAXBILL/right_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=False,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right, view ='medial',axes=ax[2])#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#surf_plot4.axes[1].text(3,.8*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
#surf_plot4.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
plot_surf_contours(fsaverage.infl_right, texture_contour, figure=surf_plot4,axes=ax[2], legend=True,levels = [1], colors=[[.3,.3,.3,1]])
#plt.savefig(os.path.join(fig_save_loc,f'COAXBILL/right_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()


# Plot correlations

In [ ]:
#"Social" alone
nodes_ind = np.where( (pval_nodes_rand[:,1]<.05))[0]
for n in nodes_ind:
    lbl = eval(shen268_lbl[str(n+1)][0])['name']
    print(n+1,lbl)

In [ ]:
#"Social" alone
nodes_ind = np.where( (pval_nodes_coaxbill<.05))[0]
for n in nodes_ind:
    lbl = eval(shen268_lbl[str(n+1)][0])['name']
    print(n+1,np.round(corr_nodes_coaxbill[n],2),np.round(np.nanmean(coef_slope_coax[:,n]-coef_slope_bill[:,n]),2),lbl)

In [ ]:
y

In [ ]:
x = "slopeReg_diff"

#for n in range(268):
n=138
#n=191 # left MTG
hemi_txt = 'Right hem' if n < 134 else 'Left hem'
lbl  = '\n'.join(eval(shen268_lbl[str(n+1)][0])['name'].split(';'))
fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_diff':coef_slope_coax[:,n]-coef_slope_bill[:,n], 'responses':(responses[:,0]==1)&(responses[:,1]==0)})
fMRI_data.set_index("Subject", inplace=True)
data = fMRI_data.join(behav_data, how='inner') # join betas and trait info


d = data.loc[data['responses']==True,:]
g = explore_correlation(x,y,d)
plt.xlabel('Beta COAX-BILL')
plt.text(plt.xlim()[0],100,f'"{resp_lbl[resp]}" (n={d.shape[0]}),\n Node{n+1}',fontweight = 'bold')
plt.xlim(min(data[x]),max(data[x]))
plt.ylim(min(data[y]),max(data[y]))
plt.text(plt.xlim()[0],120,f'"{resp_lbl[resp]}",{hemi_txt}\n{lbl}',fontweight='bold')
#g.savefig(f'../results/RandBeta{n+1}_vs_loneliness.png')



In [ ]:
np.where(pval_nodes_fish[:,0]<.05)[0]

In [ ]:
np.where(pval_nodes_fish[:,1]<.05)[0]

In [ ]:
np.where(pval_nodes_rand[:,1]<.05)

In [ ]:
nodes_ind =np.where((pval_nodes_rand[:,1]<.05) & (pval_nodes_fish[:,0]<.05))[0]
for n in nodes_ind:
    lbl = eval(shen268_lbl[str(n+1)][0])['name']
    print(n+1,lbl)

In [ ]:
np.where((pval_nodes_rand[:,1]<.05) & (pval_nodes_fish[:,1]<.05))

In [ ]:
x = "slopeReg_node"

#for n in range(268):
n=84 
hemi_txt = 'Right hem' if n < 134 else 'Left hem'
lbl  = '\n'.join(eval(shen268_lbl[str(n+1)][0])['name'].split())
fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_node':coef_slope_rand[:,n], 'responses':responses[:,4]})
fMRI_data.set_index("Subject", inplace=True)
data = fMRI_data.join(behav_data, how='inner') # join betas and trait info

for resp in [0,1]:
    d = data.loc[data['responses']==resp,:]
    g = explore_correlation(x,y,d,'Spearman')
    plt.xlabel('Beta RANDOM MECH\n(<<< Deactivation   Activation >>>)')
    plt.text(plt.xlim()[0],100,f'"{resp_lbl[resp]}" (n={d.shape[0]}),\n Node{n+1}',fontweight = 'bold')
    plt.xlim(min(data[x]),max(data[x]))
    plt.ylim(min(data[y]),max(data[y]))
    if resp ==0:
        plt.text(plt.xlim()[0],120,f'"{resp_lbl[resp]}",{hemi_txt}\n{lbl}',fontweight='bold')
    else:
        plt.text(plt.xlim()[0],120,f'"{resp_lbl[resp]}"',fontweight='bold')
    #g.savefig(f'../results/RandBeta{n+1}_vs_loneliness.png')



In [ ]:
x = "slopeReg_node"
#for n in range(268):
n=84
hemi_txt = 'Right hem' if n < 134 else 'Left hem'
lbl  = '\n'.join(eval(shen268_lbl[str(n+1)][0])['name'].split())
fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_node':coef_slope_fish[:,n], 'responses':responses[:,3]})
fMRI_data.set_index("Subject", inplace=True)
data = fMRI_data.join(behav_data, how='inner') # join betas and trait info

for resp in [0,1]:
    d = data.loc[data['responses']==resp,:]
    g = explore_correlation(x,y,d,'Spearman')
    plt.xlabel('Beta FISHING\n(<<< Deactivation   Activation >>>)')
    plt.text(plt.xlim()[0],100,f'"{resp_lbl[resp]}" (n={d.shape[0]}),\n Node{n+1}',fontweight = 'bold')
    plt.xlim(min(data[x]),max(data[x]))
    plt.ylim(min(data[y]),max(data[y]))
    if resp ==0:
        plt.text(plt.xlim()[0],120,f'"{resp_lbl[resp]}",{hemi_txt}\n{lbl}',fontweight='bold')
    else:
        plt.text(plt.xlim()[0],120,f'"{resp_lbl[resp]}"',fontweight='bold')
    #g.savefig(f'../results/RandBeta{n+1}_vs_loneliness.png')



In [ ]:
nodes = np.zeros((268,))
nodes[n] = corr_nodes[n,1]
cmap = 'RdBu_r'
img = roi_to_brain(pd.Series(nodes), mask_x)
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= True,threshold = .0001)
ax_plot.title(title_txt,fontsize=24,bgcolor='k',color='w',fontweight='bold')
ax_plot._colorbar_ax.text(1,.8*vmax,txt,fontsize=20,fontdict = {'verticalalignment':'top','rotation':0})#get_legend()

#RH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
surf_plot3.axes[1].text(3,.8*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
surf_plot3.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')

surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right,view='medial')#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})


  

In [ ]:
nodes = np.zeros((268,))
nodes[pval_nodes[:,1]<.05] = corr_nodes[pval_nodes[:,1]<.05,1]
    
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
surf_plot3.axes[1].text(3,.8*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
surf_plot3.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')

   

# plot timecourses

In [ ]:
data_file_loc

In [ ]:
tr0_ind = 3 # ind at which tr=0 starts
[timepts_indiv_movie,vid_start_rel_tr] = np.load(os.path.join(data_file_loc,'Video_TRs.npy'),allow_pickle=True)
print(vid_start_rel_tr)
l_task = 28

def remove_pretrial_TRs(tcs,vid_start_rel_tr,pretrial_TRs):

    tcs1 = np.empty((tcs.shape[0],tcs.shape[1],28+pretrial_TRs,tcs.shape[3]))
    tcs1[:] = np.nan

    for vid_no in range(10):
        tr0 = vid_start_rel_tr[vid_no]
        tcs1[:,:,:,vid_no] = tcs[:,:,tr0-pretrial_TRs:tr0+28,vid_no] # 31 timepts
    return tcs1

In [ ]:
#select subs who responded "social" to the yes movie and nonsocial to the no movie

print('\ntrial-wise norm data.')
fileName = os.path.join(data_file_loc,'timecourses_run_norm','timecourse-all-movies_zscorenorm.npy') # load fmri data
tcs_run_z = np.load(fileName) # nsubs * nnodes * ntimepts *nmovies
print('before:',tcs_run_z.shape)
tcs_run_z = remove_pretrial_TRs(tcs_run_z,vid_start_rel_tr,tr0_ind)
print('after:',tcs_run_z.shape)

In [ ]:
nodes = np.where(pval_nodes[:,1]<.05)[0]
nodes

In [ ]:
# get the right red/blue shades
#red_rgb = [103,0,31] # edges of RdBu
#blue_rgb = [5,48,97] # edges of RdBu
red_rgb =[188,61,62] # from Emily
blue_rgb = [54,122,177] # from Emily
red_rgb = np.array(red_rgb)/255
blue_rgb = np.array(blue_rgb)/255
alpha = .2 # transparency inside boxplots, for datapts etc.

In [ ]:
fMRI_data = pd.DataFrame({'Subject':sub_id_all,'sub_ind':np.arange(0,1048),'slopeReg_node':coef_slope_rand[:,n], 'responses':responses[:,4]})
fMRI_data.set_index("Subject", inplace=True)
data = fMRI_data.join(behav_data, how='inner') # join betas and trait info


In [ ]:
fMRI_data


In [ ]:
sorted_data = data.loc[data['responses']==1,['slopeReg_node','ASR_Intn_T','sub_ind','responses']]
sorted_data = sorted_data.sort_values(by=['ASR_Intn_T'])
sorted_data

In [ ]:
movie_ind = 4
ymin,ymax = -5,5
n = 58
subs = sorted_data['sub_ind'].values

ncols = 15
nrows= int(np.ceil(len(subs)/ncols))
#fig, axes = plt.subplots(67,4, figsize=(15,150))
xwidth = 30
yheight = 20# .75*nrows*xwidth/ncols
fig, ax = plt.subplots(nrows, ncols, sharex=True, sharey=True, figsize = (xwidth,yheight))
fig.set_figwidth(xwidth)
fig.set_figheight(yheight)

TR=.72
xlbls = [-2,0,5,10,15,20]
x_ticks = [(i/TR)+tr0_ind for i in xlbls]
xlbls = [str(i) for i in xlbls]
lbls = [str(np.round((i-tr0_ind)*TR,1)) for i in x_ticks]

plt.suptitle(f'Node {n+1},{eval(shen268_lbl[str(n+1)][0])["name"]}',fontweight='bold')

for i,s in enumerate(subs):

    r = int(np.floor(i/ncols))
    c = int(np.floor(i%ncols))

    movieTC_yes = tcs_run_z[s,n,:,movie_ind] # nsubs * ntimepts
    #movieTC_no  = tcs_run_z[responses[:,movie_ind]==0,n,:,movie_ind] # nsubs * ntimepts
    #sig = np.where(~np.isnan(meanDiff_rand[n,:]))[0]    

    ax[r,c].set_ylim(ymin,ymax)
    ax[r,c].plot(range(movieTC_yes.shape[0]),movieTC_yes,color = 'k',linewidth=2,label ='"Social"')
    #ax[r,c].errorbar(range(movieTC_yes.shape[1]),np.nanmean(movieTC_no,axis=0),stats.sem(movieTC_no,axis=0,nan_policy='omit'),color = blue_rgb,linewidth=2,label ='"Non-social"')
    #if r==0:
    if sorted_data["slopeReg_node"][i] > 1:
        ax[r,c].set_title(f'Sub {s},\nasr={int(sorted_data["ASR_Intn_T"][i])},b={sorted_data["slopeReg_node"][i]:.2f}', color='r',fontsize = 16)
    elif sorted_data["slopeReg_node"][i] > .5:
        ax[r,c].set_title(f'Sub {s},\nasr={int(sorted_data["ASR_Intn_T"][i])},b={sorted_data["slopeReg_node"][i]:.2f}', color=[.5,.25,.25],fontsize = 16,)
    elif sorted_data["slopeReg_node"][i] < -1:
        ax[r,c].set_title(f'Sub {s},\nasr={int(sorted_data["ASR_Intn_T"][i])},b={sorted_data["slopeReg_node"][i]:.2f}', color='b',fontsize = 16)
    elif sorted_data["slopeReg_node"][i] < -.5:
        ax[r,c].set_title(f'Sub {s},\nasr={int(sorted_data["ASR_Intn_T"][i])},b={sorted_data["slopeReg_node"][i]:.2f}', color=[.25,.25,.5],fontsize = 16)
    else:
        ax[r,c].set_title(f'Sub {s},\nasr={int(sorted_data["ASR_Intn_T"][i])},b={sorted_data["slopeReg_node"][i]:.2f}',fontsize = 16)
    
    ax[r,c].vlines(tr0_ind,-1,2,color=[.8,.8,.8])
    ax[r,c].hlines(0,0,31,color=[.8,.8,.8])
    ax[r,c].set_xticks(x_ticks)
    ax[r,c].set_xticklabels(xlbls)

ax[r,c+1].axis('off')
ax[r,c+2].axis('off')

plt.tight_layout()

In [ ]:
sorted_data['ASR_Intn_T'][i]

In [ ]:
y = "ASR_Extn_T"
g = explore_correlation(x, y)
plt.xlabel('Beta Rand')


Determine whether the difference in correlation between `Soc-NonSoc_pc` and internalizing vs externalizing symptoms is statistically significant:

In [ ]:
x = "Soc-NonSoc_pc"
y = "ASR_Intn_T"
z = "ASR_Extn_T"

inds = ~np.isnan(data[x]) & ~np.isnan(data[y]) & ~np.isnan(data[z])
print(inds.sum())

rs_xy = stats.spearmanr(data[x][inds], data[y][inds])[0]
rs_xz = stats.spearmanr(data[x][inds], data[z][inds])[0]
rs_yz = stats.spearmanr(data[y][inds], data[z][inds])[0]
print(rs_xy)
print(rs_xz)
print(rs_yz)

The following function was taken from here: https://github.com/psinger/CorrelationStats/blob/master/corrstats.py

In [ ]:
from scipy.stats import t, norm
from math import atanh, pow
from numpy import tanh

def rz_ci(r, n, conf_level = 0.95):
    zr_se = pow(1/(n - 3), .5)
    moe = norm.ppf(1 - (1 - conf_level)/float(2)) * zr_se
    zu = atanh(r) + moe
    zl = atanh(r) - moe
    return tanh((zl, zu))

def rho_rxy_rxz(rxy, rxz, ryz):
    num = (ryz-1/2.*rxy*rxz)*(1-pow(rxy,2)-pow(rxz,2)-pow(ryz,2))+pow(ryz,3)
    den = (1 - pow(rxy,2)) * (1 - pow(rxz,2))
    return num/float(den)

def dependent_corr(xy, xz, yz, n, twotailed=True, conf_level=0.95, method='steiger'):
    """
    Calculates the statistic significance between two dependent correlation coefficients
    @param xy: correlation coefficient between x and y
    @param xz: correlation coefficient between x and z
    @param yz: correlation coefficient between y and z
    @param n: number of elements in x, y and z
    @param twotailed: whether to calculate a one or two tailed test, only works for 'steiger' method
    @param conf_level: confidence level, only works for 'zou' method
    @param method: defines the method uses, 'steiger' or 'zou'
    @return: t and p-val
    """
    if method == 'steiger':
        d = xy - xz
        determin = 1 - xy * xy - xz * xz - yz * yz + 2 * xy * xz * yz
        av = (xy + xz)/2
        cube = (1 - yz) * (1 - yz) * (1 - yz)

        t2 = d * np.sqrt((n - 1) * (1 + yz)/(((2 * (n - 1)/(n - 3)) * determin + av * av * cube)))
        p = 1 - t.cdf(abs(t2), n - 3)

        if twotailed:
            p *= 2

        return t2, p
    elif method == 'zou':
        L1 = rz_ci(xy, n, conf_level=conf_level)[0]
        U1 = rz_ci(xy, n, conf_level=conf_level)[1]
        L2 = rz_ci(xz, n, conf_level=conf_level)[0]
        U2 = rz_ci(xz, n, conf_level=conf_level)[1]
        rho_r12_r13 = rho_rxy_rxz(xy, xz, yz)
        lower = xy - xz - pow((pow((xy - L1), 2) + pow((U2 - xz), 2) - 2 * rho_r12_r13 * (xy - L1) * (U2 - xz)), 0.5)
        upper = xy - xz + pow((pow((U1 - xy), 2) + pow((xz - L2), 2) - 2 * rho_r12_r13 * (U1 - xy) * (xz - L2)), 0.5)
        return lower, upper
    else:
        raise Exception('Wrong method!')

In [ ]:
n = inds.sum()

xy = stats.spearmanr(data[x][inds], data[y][inds])[0]
xz = stats.spearmanr(data[x][inds], data[z][inds])[0]
yz = stats.spearmanr(data[y][inds], data[z][inds])[0]

dependent_corr(xy, xz, yz, n, twotailed=False)

Look at responses on RANDMECH:

In [ ]:
all_trial_data = pd.read_csv('../data/hcp_social_826subs.csv')
all_trial_data.set_index("subj_idx", inplace=True)
all_trial_data.index.rename("Subject", inplace=True)
all_trial_data.index = all_trial_data.index.map(str)
all_trial_data.head()

In [ ]:
data = data.join(all_trial_data[all_trial_data["movie"]=="Random mechanical.AVI"]["response"])
data.rename(columns={"response": "rand_mech_response"}, inplace=True)
data.head()

In [ ]:
#red_rgb = [103,0,31] # edges of RdBu
#blue_rgb = [5,48,97] # edges of RdBu
red_rgb =[188,61,62] # from Emily
blue_rgb = [54,122,177] # from Emily
red_rgb = np.array(red_rgb)/255
blue_rgb = np.array(blue_rgb)/255
alpha = .2 # transparency inside boxplots, for datapts etc.

In [ ]:
#colors = [red_rgb,blue_rgb]
# Set your custom color palette
#myPalette = sns.set_palette(sns.color_palette(colors))

In [ ]:
list(col[:2])+[alpha]

In [ ]:
# Create a new variable denoting subjects who responded either "yes" or "unsure"
data["rand_mech_yesorunsure"] = data["rand_mech_response"] > 0

y = "ASR_Intn_T"

fig, ax = plt.subplots(figsize=(3,10))

#sns.set_palette(["tab:blue", "tab:red"])
ax = sns.boxplot(x="rand_mech_yesorunsure", y=y, data=data)
ax.set_xlabel("RANDMECH response")
ax.set_xticklabels(["\"Non-\nsocial\"", "\"Social\" or \n\"Unsure\""])

'''for patch in ax.artists:
 r, g, b, a = patch.get_facecolor()
 patch.set_facecolor((r, g, b, alpha))
 patch.set_edgecolor((r, g, b, 1))'''
 
for i,artist in enumerate(ax.artists):
    if i == 0:
        col = blue_rgb + [alpha]
    else:
        col = red_rgb + [alpha]
        # Set the linecolor on the artist to the facecolor, and set the facecolor to None
    #col = artist.get_facecolor()
    artist.set_edgecolor(col)
    #artist.set_edgecolor('None')
    artist.set_facecolor(list(col[:3])+[alpha])
    if np.round(col[0],2) ==  .75:
        print(i)
    # Each box has 6 associated Line2D objects (to make the whiskers, fliers, etc.)
    # Loop over them here, and use the same colour as above
    
    for j in range(i*6,i*6+6):
        line = ax.lines[j]
        if j in [4,10]:
            line.set_color('k')
            line.set_linestyle('--')
            line.set_linewidth(2)
        else:
            line.set_color(col)
        line.set_mfc(col)
        line.set_mec(col)
        if j in [5,11]:
            line.set_marker('o')
plt.xlabel('')
plt.title('RANDOM MECH', fontweight = 'bold')
t, p = stats.ttest_ind(data[data["rand_mech_yesorunsure"]==1][y], data[data["rand_mech_yesorunsure"]==0][y], nan_policy='omit')
ax.annotate(f't = {t:.2f},\n(p = {p:.2g})', xy=(0.05, 0.85), xycoords='axes fraction')

fig.savefig('../results/RANDMECHresp_vs_ASRIntnT.png')


In [ ]:
def box_plot(data, edge_color, fill_color,pos,v):
    # customize boxplots
    alpha= .2
    data = data[~np.isnan(data)]
    bp = ax.boxplot(data, positions = [pos], patch_artist=True,widths=.6,vert=v,flierprops = dict(markeredgecolor=edge_color,
    markerfacecolor=fill_color, alpha=alpha))
    
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'caps']:
        plt.setp(bp[element], color=edge_color)
    for element in ['medians']:
        plt.setp(bp[element], color='k',linewidth=2,ls='dashed')

    for patch in bp['boxes']:
        patch.set(facecolor=fill_color,alpha=alpha)#'w'       
        
    return bp

In [ ]:
#data1 = data.loc[data['rand_mech_response']!=9,:]
#data1.shape
fig, ax = plt.subplots(figsize=(3,5))

y = "ASR_Intn_T"
#x = "rand_mech_response"
x = "rand_mech_yesorunsure"
rows = data[x] == 0
bp2 = box_plot(data.loc[rows,y], blue_rgb, blue_rgb,0,True)
rows = data[x] == 1
bp1 = box_plot(data.loc[rows,y], red_rgb, red_rgb,1,True)
plt.xticks(range(2),['"Non-social"','"Social"/\n "Unsure"'])
t, p = stats.ttest_ind(data[data[x]==1][y], data[data[x]==0][y], nan_policy='omit')
plt.ylabel('ASR_Intn_T')
ax.annotate(f't = {t:.2f}\n(p = {p:.2g})', xy=(0.05, 0.85), xycoords='axes fraction')
plt.title('RANDOM MECH',fontweight='bold')
fig.savefig('../results/RANDMECHresp_vs_ASRIntnT.png')


In [ ]:
# Create a new variable denoting subjects who responded either "yes" or "unsure"
data["rand_mech_yesorunsure"] = data["rand_mech_response"] > 0

y = "ASR_Intn_T"

fig, ax = plt.subplots(figsize=(5,10))

sns.set_palette(["tab:blue", "tab:red"])
ax = sns.boxplot(x="rand_mech_yesorunsure", y=y, data=data)
ax.set_xlabel("RANDMECH response")
ax.set_xticklabels(["\"Non-social\"", "\"Social\" or \n\"Unsure\""])

t, p = stats.ttest_ind(data[data["rand_mech_yesorunsure"]==1][y], data[data["rand_mech_yesorunsure"]==0][y], nan_policy='omit')
ax.annotate(f't = {t:.2f} (p = {p:.2g})', xy=(0.05, 0.9), xycoords='axes fraction')

#fig.savefig('../results/RANDMECHresp_vs_ASRIntnT.png')


Look at RT on RANDMECH:

In [ ]:
data = data.join(all_trial_data[all_trial_data["movie"]=="Random mechanical.AVI"]["rt"])
data = data.rename(columns={"rt": "rand_mech_rt"})
data.head()

In [ ]:
g = explore_correlation(x="rand_mech_rt", y="ASR_Intn_T", data=data)

In [ ]:
# Restrict correlation to only subjects that responded "yes" or "unsure"

g = explore_correlation(x="rand_mech_rt", y="ASR_Intn_T", data=data[data["rand_mech_yesorunsure"]==1])

# extra code

In [ ]:

'''# model 1: factors: traits, responses, their interaction
coef_nodes_rand_lr_model1 = []
pval_nodes_rand_lr_model1 = []

# model 2: factor: responses
coef_nodes_rand_lr_model2 = []  
pval_nodes_rand_lr_model2  = []'''

# model 3: factor: responses, traits (no interactions)
coef_nodes_fish_lr_model3 = []  
pval_nodes_fish_lr_model3  = []

print_term = True
for n in nodes_traits[1]: # FISHING 
    if n% 50 ==0:
        print('node',n)

    fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_node':coef_slope_rand[:,n], 'responses':responses[:,4]})
    fMRI_data.set_index("Subject", inplace=True)
    data = fMRI_data.join(behav_data, how='inner') # join betas and trait info
    data.reset_index(inplace=True)
    data = data.loc[:,['Subject','slopeReg_node','responses','ASR_Intn_T']]
    
    d = data.loc[(data['responses']==0)|(data['responses']==1),:]
    ind = (~np.isnan(d['slopeReg_node'])) & (~np.isnan(d['ASR_Intn_T'])) & (~np.isnan(d['responses']))
    d = d.loc[ind,:]
    
    '''model1 = Lm("slopeReg_node ~  responses", data=d)
    model1.fit(summary = False, verbose = False)
    
    model2 = Lm("slopeReg_node ~  responses + ASR_Intn_T", data=d)
    model2.fit(summary = False, verbose = False)'''

    model3 = Lm("slopeReg_node ~  responses + ASR_Intn_T + responses*ASR_Intn_T", data=d)
    model3.fit(summary = False, verbose = False)
    #model.fit(factors={"response":["Social","Unsure"]},summary = False, verbose = False) # Unsure > Social
    

    if print_term == True:
        print_term = False
        print('node_ind',n)
        '''print(model1.fit())#factors={"response":["Social","Unsure"]}))
        print(model2.fit())'''
        print(model3.fit())

    '''coef_nodes_rand_lr_model1.append([n] + [model1.coefs['Estimate'][1]] )
    pval_nodes_rand_lr_model1.append([n] + [model1.coefs['P-val'][1]]    )
    
    coef_nodes_rand_lr_model2.append([n] + [model2.coefs['Estimate'][i] for i in np.arange(1,3)])
    pval_nodes_rand_lr_model2.append([n] + [model2.coefs['P-val'][i] for i in np.arange(1,3)])'''
    
    #intercept,coef_resp,coef_trait,interaxn_resp_trait = [model3.coefs['Estimate'][i] for i in range(4)]
    coef_nodes_fish_lr_model3.append([n] + [model3.coefs['Estimate'][i] for i in np.arange(1,4)])
    pval_nodes_fish_lr_model3.append([n] + [model3.coefs['P-val'][i] for i in np.arange(1,4)])
    
coef_nodes_fish_lr_model3,pval_nodes_fish_lr_model3 = [np.array(i) for i in [coef_nodes_fish_lr_model3,pval_nodes_fish_lr_model3]]
'''coef_nodes_rand_lr_model1,pval_nodes_rand_lr_model1,coef_nodes_rand_lr_model2,pval_nodes_rand_lr_model2,\
coef_nodes_rand_lr_model3,pval_nodes_rand_lr_model3 = [np.array(i) for i in [coef_nodes_rand_lr_model1,pval_nodes_rand_lr_model1,coef_nodes_rand_lr_model2,pval_nodes_rand_lr_model2,\
coef_nodes_rand_lr_model3,pval_nodes_rand_lr_model3]]'''

#pval_nodes_fdr_rand_lr_model2 = []
pval_nodes_fdr_fish_lr_model3 = []

for ind in range(3):
    pval_nodes_fdr_fish_lr_model3.append(lsu(pval_nodes_fish_lr_model3[:,ind+1],q=.05))


sig_interaction_node_inds_fish =  [int(i) for i in np.where(pval_nodes_fdr_fish_lr_model3[2])[0]]
print(sig_interaction_node_inds_fish)
fish_sig_interaction_nodes = np.array([nodes_traits[0][sig_interaction_node_inds_fish],coef_nodes_fish_lr_model3[sig_interaction_node_inds_fish,3]]).T
fish_sig_interaction_nodes